# Computation with universal functions
---------------------

* NumPy provides an easy and flexible interface to optimized computation with arrays of data.
* Computation on NumPy arrays can be very fast using *vectorized* operations, generally implemented through NumPy's **universal functions** (**ufuncs**).
* Most common and useful arithmetic ufuncs available in the NumPy package.

### 1. Slowness of loops
-----------------
Python's default implementation (CPython) does some operations very slowly.
This is in part due to the dynamic, interpreted nature of the language: the fact that types are flexible, so that sequences of operations cannot be compiled down to efficient machine code.

The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated – for instance looping over arrays to operate on each element.
For example, computing the reciprocal of each element in array of values :

In [ ]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

For a large input this operation is very slow:

In [ ]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

The bottleneck here is not the operations themselves, but the type-checking and function dispatches that CPython must do at each cycle of the loop.

### 2. Introducing UFuncs
-----------------------
NumPy provides a **vectorized** operations -- a convenient interface into just this **kind of statically typed**, compiled routine. 

This vectorized approach is designed to push the loop into the **compiled layer** that underlies NumPy.

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

Looking at the execution time for our big array, we see that it completes orders of magnitude faster than the Python loop:

In [ ]:
%timeit (1.0 / big_array)

Vectorized operations are implemented via **ufuncs**, whose main purpose is to quickly execute repeated operations on values in NumPy arrays.

Ufuncs are extremely flexible:

In [ ]:
np.arange(5) / np.arange(1, 6)

And ufunc operations can also act on multi-dimensional arrays as well:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

* Computations using vectorization through ufuncs are nearly always more efficient than their counterpart implemented using Python loops, especially as the arrays grow in size.
* Any time you see such a loop in a Python script, you should consider whether it can be replaced with a vectorized expression.

### 3. Exploring UFuncs
------------------------
Ufuncs exist in two flavors: 
* *unary ufuncs*, which operate on a single input
* *binary ufuncs*, which operate on two inputs.

#### 3.1. Array arithmetic

* ufuncs make use of Python's native arithmetic operators

In [ ]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

*  unary ufunc for negation, and a ``**`` operator for exponentiation, and a ``%`` operator for modulus:

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

* The standard order of operations is respected:

In [ ]:
-(0.5*x + 1) ** 2

* Each of these arithmetic operations are simply convenient wrappers around specific functions built into NumPy; for example, the ``+`` operator is a wrapper for the ``add`` function:

In [ ]:
np.add(x, 2)

* The arithmetic operators :

| Operator	    | Equivalent ufunc    | Description                           |Example |
|---------------|---------------------|---------------------------------------|--------|
|``+``          |``np.add``           |Addition          |``1 + 1 = 2``|
|``-``          |``np.subtract``      |Subtraction       |``3 - 2 = 1``|
|``-``          |``np.negative``      |Unary negation         |``-2``|
|``*``          |``np.multiply``      |Multiplication    |``2 * 3 = 6``
|``/``          |``np.divide``        |Division        |``3 / 2 = 1.5``
|``//``         |``np.floor_divide``  |Floor division   |``3 // 2 = 1``
|``**``         |``np.power``         |Exponentiation    |``2 ** 3 = 8``
|``%``          |``np.mod``           |Modulus/remainder |``9 % 4 = 1``

* Additionally there are Boolean/bitwise operators.

#### 3.2. Absolute value

Just as NumPy understands Python's built-in arithmetic operators, it also understands Python's built-in absolute value function:

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

The corresponding NumPy ufunc is ``np.absolute``, which is also available under the alias ``np.abs``:

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

This ufunc can also handle complex data, in which the absolute value returns the magnitude:

In [ ]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

#### 3.3. Trigonometric functions


* Computing trigonometric functions on ndarrays:

In [ ]:
theta = np.linspace(0, np.pi, 3)

In [ ]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

The values are computed to within machine precision, which is why values that should be zero do not always hit exactly zero.

* Inverse trigonometric functions

In [ ]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

#### 3.4. Exponents and logarithms

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

The basic ``np.log`` gives the natural logarithm; ``np.log2`` and ``np.log10`` compute  base-2 and base-10 logarithms:

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

There are also some specialized versions that are useful for maintaining precision with very small input:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

When ``x`` is very small, these functions give more precise values than if the raw ``np.log`` or ``np.exp`` were to be used.

#### 3.5. Specialized ufuncs
__________________

* NumPy has many more ufuncs available, including hyperbolic trig functions, bitwise arithmetic, comparison operators, conversions from radians to degrees, rounding and remainders, and much more.

* Another excellent source for more specialized and obscure ufuncs is the submodule ``scipy.special``.